## Libraries and Data Setup

In [153]:
import sagemaker
import boto3
from sagemaker.sklearn.estimator import SKLearn


import joblib
import pathlib
from io import StringIO
import argparse
import os

import pandas as pd
import numpy as np

import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, f1_score
from sklearn.model_selection import train_test_split

In [154]:

# Specify the region
region = 'us-west-2'  # replace with your desired region

# Connect to SageMaker
boto_session = boto3.Session(region_name=region)
sm_boto3 = boto_session.client("sagemaker")
session = sagemaker.Session(boto_session=boto_session)
bucket = 'sagemaker-us-west-2-730335213049'

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


In [155]:
df = pd.read_csv('train.csv')

## Exploratory Analysis

In [156]:
df.shape

(2000, 21)

In [157]:
df.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1


In [158]:
df.describe()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
count,2000.000000,2000.0000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,...,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000
mean,1238.518500,0.4950,1.522250,0.509500,4.309500,0.521500,32.046500,0.501750,140.249000,4.520500,...,645.108000,1251.515500,2124.213000,12.306500,5.767000,11.011000,0.761500,0.503000,0.507000,1.500000
std,439.418206,0.5001,0.816004,0.500035,4.341444,0.499662,18.145715,0.288416,35.399655,2.287837,...,443.780811,432.199447,1084.732044,4.213245,4.356398,5.463955,0.426273,0.500116,0.500076,1.118314
min,501.000000,0.0000,0.500000,0.000000,0.000000,0.000000,2.000000,0.100000,80.000000,1.000000,...,0.000000,500.000000,256.000000,5.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000
25%,851.750000,0.0000,0.700000,0.000000,1.000000,0.000000,16.000000,0.200000,109.000000,3.000000,...,282.750000,874.750000,1207.500000,9.000000,2.000000,6.000000,1.000000,0.000000,0.000000,0.750000
50%,1226.000000,0.0000,1.500000,1.000000,3.000000,1.000000,32.000000,0.500000,141.000000,4.000000,...,564.000000,1247.000000,2146.500000,12.000000,5.000000,11.000000,1.000000,1.000000,1.000000,1.500000
75%,1615.250000,1.0000,2.200000,1.000000,7.000000,1.000000,48.000000,0.800000,170.000000,7.000000,...,947.250000,1633.000000,3064.500000,16.000000,9.000000,16.000000,1.000000,1.000000,1.000000,2.250000
max,1998.000000,1.0000,3.000000,1.000000,19.000000,1.000000,64.000000,1.000000,200.000000,8.000000,...,1960.000000,1998.000000,3998.000000,19.000000,18.000000,20.000000,1.000000,1.000000,1.000000,3.000000


In [159]:
df.columns

Index(['battery_power', 'blue', 'clock_speed', 'dual_sim', 'fc', 'four_g',
       'int_memory', 'm_dep', 'mobile_wt', 'n_cores', 'pc', 'px_height',
       'px_width', 'ram', 'sc_h', 'sc_w', 'talk_time', 'three_g',
       'touch_screen', 'wifi', 'price_range'],
      dtype='object')

In [160]:
df['price_range'].value_counts(normalize=True)

price_range
1    0.25
2    0.25
3    0.25
0    0.25
Name: proportion, dtype: float64

In [161]:
# Percentage of values that are null
df.isnull().mean() * 100

battery_power    0.0
blue             0.0
clock_speed      0.0
dual_sim         0.0
fc               0.0
four_g           0.0
int_memory       0.0
m_dep            0.0
mobile_wt        0.0
n_cores          0.0
pc               0.0
px_height        0.0
px_width         0.0
ram              0.0
sc_h             0.0
sc_w             0.0
talk_time        0.0
three_g          0.0
touch_screen     0.0
wifi             0.0
price_range      0.0
dtype: float64

In [162]:
features = list(df)
lables = features.pop()
x, y = df[features], df[lables]

In [163]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=.2, random_state=0)

In [164]:
print('X_train shape:  ', X_train.shape)
print('X_test shape:  ', X_test.shape)
print('Y_train shape:  ', y_train.shape)
print('Y_test shape:  ', y_test.shape)

X_train shape:   (1600, 20)
X_test shape:   (400, 20)
Y_train shape:   (1600,)
Y_test shape:   (400,)


In [165]:
X_train.to_csv('split_data/x_train.csv', index=False)
X_test.to_csv('split_data/x_test.csv', index=False)

### Push Split Data To S3 Bucket

In [166]:
sk_prefix = f'sagemaker/{bucket}/sklearncontainer'
train_path = session.upload_data(
    path='split_data/x_train.csv',
    bucket=bucket,
    key_prefix=sk_prefix
)

test_path = session.upload_data(
    path='split_data/x_test.csv',
    bucket=bucket,
    key_prefix=sk_prefix
)

print(train_path)
print(test_path)

s3://sagemaker-us-west-2-730335213049/sagemaker/sagemaker-us-west-2-730335213049/sklearncontainer/x_train.csv
s3://sagemaker-us-west-2-730335213049/sagemaker/sagemaker-us-west-2-730335213049/sklearncontainer/x_test.csv


In [167]:
%%writefile script.py
import sagemaker
import boto3

import joblib
import pathlib
from io import StringIO
import argparse
import os

import pandas as pd
import numpy as np

import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, f1_score
from sklearn.model_selection import train_test_split

def model_fxn(model_dir):
    clf = joblib.load(os.path.join(model_dir, 'model.joblib'))
    return clf

if __name__ == '__main__':
    print('[INFO] Extracting arguments')
    parser = argparse.ArgumentParser()

    """
        Hyperparameters sent by client are passed as command lin arguments to the script
    """
    parser.add_argument("--n_estimators", type=int, default=100)
    parser.add_argument("--random_state", type=int, default=0)

    # Data, model and output directories
    parser.add_argument("--model-dir", type=str, default=os.environ.get("SM_MODEL_DIR"))
    parser.add_argument("--train", type=str, default=os.environ.get("SM_CHANNEL_TRAIN"))
    parser.add_argument("--test", type=str, default=os.environ.get("SM_CHANNEL_TEST"))
    parser.add_argument("--train-file", type=str, default=os.environ.get("x_train.csv"))
    parser.add_argument("--test-file", type=str, default=os.environ.get("x_test.csv"))

    args, _ = parser.parse_known_args()

    print("SKLearn Version:  ", sklearn.__version__)
    print("Joblib Version:  ", joblib.__version__)

    print("[INFO] Reading Data")
    print()
    train_df = pd.read_csv(os.path.join(args.train, args.train_file))
    test_df = pd.read_csv(os.path.join(args.test, args.test_file))

    features = list(train_df.columns)
    labels = features.pop()

    print("Building training and testing datasets")
    print()

    X_train = train_df[features]
    y_train = train_df[labels]
    X_test = test_df[features]
    y_test = test_df[labels]

    print('Column order:  ')
    print(features)
    print()

    print('Label Columns:  ', labels)
    print()

    print('Data shape:  ')
    print()
    print('---- Shape of Training Data (80%) ----')
    print(X_train.shape)
    print(y_train.shape)

    print()
    print('---- Shape of Testing Data (20%) ----')
    print(X_test.shape)
    print(y_test.shape)
    print()

    print('Training Random Forest Model')
    print()

    model = RandomForestClassifier(n_estimators=args.n_estimators, random_state=args.random_state)
    model.fit(X_train, y_train)
    print()

    model_path = os.path.join(args.model_dir, "model.joblib")
    joblib.dump(model, model_path)
    print('Model persisted at ', model_path)
    print()

    y_pred_test = model.predict(X_test)
    test_accuracy = accuracy_score(y_test, y_pred_test)
    test_report = classification_report(y_test, y_pred_test)

    print()
    print('---- Metrics Results For Testing Data ----')
    print()
    
    print('Total Rows:  ', X_test.shape[0])
    print('[TESTING] Model Accuracy is:  ', test_accuracy)
    print('[TESTING] Testing Report:  ')
    print(test_report)



Overwriting script.py


In [168]:
FRAMEWORK_VERSION = '0.23-1'

sklearn_estimator = SKLearn(
    entry_point='script.py',
    role='arn:aws:iam::730335213049:role/service-role/AmazonSageMaker-ExecutionRole-20240608T194780',
    instance_count=1,
    instance_type='ml.m5.large',
    framework_version=FRAMEWORK_VERSION,
    base_job_name='RF-custom-sklearn',
    hyperparameters={
        'n_estimators': 100,
        'random_state': 0
    },
    use_spot_instances=True,
    max_wait=7200,
    max_run=3600,
    sagemaker_session=session
)

In [169]:
# sklearn_estimator.latest_training_job.wait(logs='None')
# artifact = sm_boto3.describe_training_job(
#     TrainingJobName=sklearn_estimator.latest_training_job.name
# )['ModelArtifacts']['S3ModelArtifacts']

# print('Model Artifact Persisted at:  ', artifact)

In [170]:
# artifact

In [171]:
# launch training job with asynchronous call
sklearn_estimator.fit({'train': train_path, 'test': test_path}, wait=True)

INFO:sagemaker:Creating training-job with name: RF-custom-sklearn-2024-06-14-06-46-08-484


2024-06-14 06:46:09 Starting - Starting the training job...
2024-06-14 06:46:23 Starting - Preparing the instances for training...
2024-06-14 06:46:51 Downloading - Downloading input data...
2024-06-14 06:47:16 Downloading - Downloading the training image...
2024-06-14 06:48:07 Training - Training image download completed. Training in progress.
2024-06-14 06:48:07 Uploading - Uploading generated training model2024-06-14 06:48:00,751 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2024-06-14 06:48:00,755 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-06-14 06:48:00,798 sagemaker_sklearn_container.training INFO     Invoking user training script.
2024-06-14 06:48:00,979 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-06-14 06:48:00,991 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-06-14 06:48:01,004 sagemaker-training-toolkit INFO

UnexpectedStatusException: Error for Training job RF-custom-sklearn-2024-06-14-06-46-08-484: Failed. Reason: AlgorithmError: framework error: 
Traceback (most recent call last):
  File "/miniconda3/lib/python3.7/site-packages/sagemaker_containers/_trainer.py", line 84, in train
    entrypoint()
  File "/miniconda3/lib/python3.7/site-packages/sagemaker_sklearn_container/training.py", line 39, in main
    train(environment.Environment())
  File "/miniconda3/lib/python3.7/site-packages/sagemaker_sklearn_container/training.py", line 35, in train
    runner_type=runner.ProcessRunnerType)
  File "/miniconda3/lib/python3.7/site-packages/sagemaker_training/entry_point.py", line 100, in run
    wait, capture_error
  File "/miniconda3/lib/python3.7/site-packages/sagemaker_training/process.py", line 291, in run
    cwd=environment.code_dir,
  File "/miniconda3/lib/python3.7/site-packages/sagemaker_training/process.py", line 208, in check_error
    info=extra_info,
sagemaker_training.errors.Execut